<a href="https://colab.research.google.com/github/AhmadYasserHamad/Assignment_4/blob/main/K_means.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install opendatasets

In [15]:
import pandas as pd
import opendatasets as od
df = pd.read_csv("data.csv")
# Quick look at the dataset
print(df.head())
print(df.info())

         id diagnosis  radius_mean  texture_mean  perimeter_mean  area_mean  \
0    842302         M        17.99         10.38          122.80     1001.0   
1    842517         M        20.57         17.77          132.90     1326.0   
2  84300903         M        19.69         21.25          130.00     1203.0   
3  84348301         M        11.42         20.38           77.58      386.1   
4  84358402         M        20.29         14.34          135.10     1297.0   

   smoothness_mean  compactness_mean  concavity_mean  concave points_mean  \
0          0.11840           0.27760          0.3001              0.14710   
1          0.08474           0.07864          0.0869              0.07017   
2          0.10960           0.15990          0.1974              0.12790   
3          0.14250           0.28390          0.2414              0.10520   
4          0.10030           0.13280          0.1980              0.10430   

   ...  texture_worst  perimeter_worst  area_worst  smoothness

In [16]:
# drop un named column in data
df = df.drop('Unnamed: 32', axis=1)
print("\nDropped 'Unnamed: 32' column (empty column)")

# Remove non-feature columns (id and diagnosis for unsupervised learning) and id have no meaning
X = df.drop(['id', 'diagnosis'], axis=1, errors='ignore')

print(f"\nRemaining columns: {X.columns.tolist()}")
print(f"Features shape: {X.shape}")

# Normalize the features using StandardScaler
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

# Convert back to DataFrame for easier viewing
normalized_df = pd.DataFrame(X_normalized, columns=X.columns)

print("\n" + "="*50)
print("NORMALIZATION COMPLETE")
print("="*50)
print(f"\nOriginal data statistics:")
print(X.describe())
print(f"\nNormalized data statistics:")
print(normalized_df.describe())

print(f"\nNormalized data shape: {X_normalized.shape}")

# Display first few rows of normalized data
print("\nFirst 5 rows of normalized data:")
print(normalized_df.head())


DATA PREPROCESSING FOR K-MEANS

Dropped 'Unnamed: 32' column (empty column)

Remaining columns: ['radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean', 'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean', 'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se', 'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se', 'fractal_dimension_se', 'radius_worst', 'texture_worst', 'perimeter_worst', 'area_worst', 'smoothness_worst', 'compactness_worst', 'concavity_worst', 'concave points_worst', 'symmetry_worst', 'fractal_dimension_worst']
Features shape: (569, 30)

NORMALIZATION COMPLETE

Original data statistics:
       radius_mean  texture_mean  perimeter_mean    area_mean  \
count   569.000000    569.000000      569.000000   569.000000   
mean     14.127292     19.289649       91.969033   654.889104   
std       3.524049      4.301036       24.298981   351.914129   
min       6.981000  

k-means ++ inilization

In [20]:
import numpy as np

def kmeans_plusplus_init(X, k, random_state=None):
    if random_state is not None:
        np.random.seed(random_state)

    n_samples, n_features = X.shape
    centers = np.zeros((k, n_features))

    # Step 1: Choose first center randomly from data points
    first_center_idx = np.random.randint(0, n_samples)
    centers[0] = X[first_center_idx]
    print(f"Center 1 initialized at index {first_center_idx}")

    # Step 2: Choose remaining k-1 centers
    for i in range(1, k):
        # Calculate distance from each point to nearest existing center
        distances = np.zeros(n_samples)

        for j in range(n_samples):
            # Calculate distances to all existing centers
            dist_to_centers = np.zeros(i)
            for c in range(i):
                dist_to_centers[c] = np.sum((X[j] - centers[c]) ** 2)

            # Store minimum distance (squared distance to nearest center)
            distances[j] = np.min(dist_to_centers)

        # Choose next center with probability proportional to distance squared
        probabilities = distances / np.sum(distances)
        next_center_idx = np.random.choice(n_samples, p=probabilities)
        centers[i] = X[next_center_idx]
        print(f"Center {i+1} initialized at index {next_center_idx}")

    print(f"\nK-means++ initialization complete!")
    print(f"Initial centers shape: {centers.shape}")

    return centers
k = 2
initial_centers = kmeans_plusplus_init(X_normalized , k=k, random_state=42)

print(f"\nInitial Centers:")
print(initial_centers)

Center 1 initialized at index 102
Center 2 initialized at index 432

K-means++ initialization complete!
Initial centers shape: (2, 30)

Initial Centers:
[[-0.5530585   0.28631105 -0.60751564 -0.55798194 -1.15503513 -1.21215527
  -0.81568788 -0.8052661  -0.2651265  -0.85447642 -0.76793904  0.64254367
  -0.83316584 -0.5644996  -0.65368614 -1.08314703 -0.70305211 -0.81090835
  -0.73522485 -0.85594642 -0.60658404  1.16641373 -0.67557894 -0.58500369
  -0.87972471 -1.05373391 -0.75651421 -0.61357437 -0.33448538 -0.84042616]
 [ 1.71905507  0.05825847  1.72302589  1.69255646  1.20552258  0.84445927
   1.56310249  1.98564279 -0.31989066 -0.32145729  0.10079781 -0.39163235
   0.07026503  0.26738504  0.68203468  0.09343955  0.78138781  1.18475716
   0.48040333  0.07946914  1.19296315 -0.09888253  1.15386726  1.05196463
   1.4961806   0.25404212  1.24069484  1.56367679  0.21232446 -0.17709882]]


k-means algo

In [23]:
def kmeans(X, k, centers, epochs, tolerance=1e-6):
    n_samples, n_features = X.shape
    labels = np.zeros(n_samples, dtype=int)
    inertia_history = []

    print(f"Starting K-means algorithm...")
    print(f"Data shape: {X.shape}")
    print(f"Number of clusters (k): {k}")
    print(f"Max epochs: {epochs}")
    print(f"Convergence tolerance: {tolerance}\n")

    for epoch in range(epochs):
        # Step 1: Assignment - Assign each point to nearest center
        for i in range(n_samples):
            distances = np.zeros(k)
            for j in range(k):
                distances[j] = np.sum((X[i] - centers[j]) ** 2)
            labels[i] = np.argmin(distances)

        # Step 2: Update - Calculate new centers
        new_centers = np.zeros((k, n_features))
        for j in range(k):
            # Get all points assigned to cluster j
            cluster_points = X[labels == j]
            if len(cluster_points) > 0:
                new_centers[j] = np.mean(cluster_points, axis=0)
            else:
                # If no points assigned, keep old center
                new_centers[j] = centers[j]

        # Calculate inertia (sum of squared distances to nearest center)
        inertia = 0
        for i in range(n_samples):
            inertia += np.sum((X[i] - centers[labels[i]]) ** 2)
        inertia_history.append(inertia)

        # Check for convergence
        center_shift = np.sum(np.abs(new_centers - centers))

        print(f"Epoch {epoch + 1}/{epochs} - Inertia: {inertia:.4f} - Center shift: {center_shift:.6f}")

        if center_shift < tolerance:
            print(f"\nConverged at epoch {epoch + 1}! (Center shift {center_shift:.6f} < tolerance {tolerance})")
            centers = new_centers
            break

        centers = new_centers

    print(f"\nK-means complete!")
    print(f"Final inertia: {inertia_history[-1]:.4f}")

    # Print cluster sizes
    print(f"\nCluster sizes:")
    for i in range(k):
        cluster_size = np.sum(labels == i)
        print(f"Cluster {i}: {cluster_size} points")

    return centers, labels, inertia_history


# Example usage
# Assuming normalized_df is available from previous code
X_data = normalized_df.values

# Parameters
k = 2
epochs = 100

# Initialize centers using K-means++
print("="*50)
print("K-MEANS++ INITIALIZATION")
print("="*50)
initial_centers = kmeans_plusplus_init(X_data, k=k, random_state=42)

# Run K-means
print("\n" + "="*50)
print("K-MEANS CLUSTERING")
print("="*50)
final_centers, labels, inertia_history = kmeans(X_data, k, initial_centers, epochs)

print("\n" + "="*50)
print("RESULTS")
print("="*50)
print(f"Final centers shape: {final_centers.shape}")
print(f"Labels shape: {labels.shape}")
print(f"Unique labels: {np.unique(labels)}")

K-MEANS++ INITIALIZATION
Center 1 initialized at index 102
Center 2 initialized at index 432

K-means++ initialization complete!
Initial centers shape: (2, 30)

K-MEANS CLUSTERING
Starting K-means algorithm...
Data shape: (569, 30)
Number of clusters (k): 2
Max epochs: 100
Convergence tolerance: 1e-06

Epoch 1/100 - Inertia: 16420.4473 - Center shift: 27.497835
Epoch 2/100 - Inertia: 11599.9686 - Center shift: 0.834528
Epoch 3/100 - Inertia: 11595.6965 - Center shift: 0.172283
Epoch 4/100 - Inertia: 11595.5266 - Center shift: 0.000000

Converged at epoch 4! (Center shift 0.000000 < tolerance 1e-06)

K-means complete!
Final inertia: 11595.5266

Cluster sizes:
Cluster 0: 375 points
Cluster 1: 194 points

RESULTS
Final centers shape: (2, 30)
Labels shape: (569,)
Unique labels: [0 1]
